In [1]:
from framework import Framework

In [2]:
data_dir = "daily_cluster_data/daily_cluster_matrices_min_6"

fw = Framework(data_dir,verbose=1)

experimenting with:
 ['louvain']


In [21]:
one_community_detection_run = fw.results['sample_0']['cooccurrence']['louvain']['louvain_0']['communities']

In [23]:
one_community_detection_run

{0: 0,
 1: 1,
 2: 1,
 3: 1,
 4: 0,
 5: 0,
 6: 1,
 7: 0,
 8: 1,
 9: 0,
 10: 1,
 11: 1,
 12: 1,
 13: 1,
 14: 1,
 15: 1,
 16: 1,
 17: 0,
 18: 1,
 19: 1,
 20: 0,
 21: 1,
 22: 1,
 23: 1,
 24: 1,
 25: 0,
 26: 1,
 27: 0,
 28: 1,
 29: 1,
 30: 0,
 31: 1,
 32: 1,
 33: 1,
 34: 1,
 35: 1,
 36: 1,
 37: 1,
 38: 1,
 39: 1,
 40: 1,
 41: 1,
 42: 0,
 43: 1,
 44: 0,
 45: 1,
 46: 0,
 47: 1,
 48: 1}

In [9]:
results

In [ ]:
from collections import Counter
from itertools import combinations
from partition_metrics import all_metrics

def compare_two_coms(s1,s2):
    coms1 = list(fw.results[f'sample_{s1}']['normalized_counts']['louvain']['louvain']['communities'].values())
    coms2 = list(fw.results[f'sample_{s2}']['normalized_counts']['louvain']['louvain']['communities'].values())

    # print(coms1)
    r = all_metrics(coms1,coms2,verbose=0)
    print('ari',r['ari'])
    print('nmi',r['nmi'])
    print('ned',r['norm_edit_distance'])
    print()

for s1,s2 in combinations(list(range(5)),2):
    compare_two_coms(s1,s2)
    # print('-'*60)

KeyError: 'louvain'

In [7]:
import pandas as pd

rows = []
for sample_ID, modeling_methods in fw.results.items():
    for modeling_method, comm_methods in modeling_methods.items():
        for comm_method_name, hp_dict in comm_methods.items():
            for hp_ID, result in hp_dict.items():
                row = {
                    'sample_ID': sample_ID,
                    'modeling_method': modeling_method,
                    'comm_method_name': comm_method_name,
                    'hp_ID': hp_ID,
                    **result  # assuming result is a dict
                }
                rows.append(row)

df = pd.DataFrame(rows)
df

,sample_ID,modeling_method,comm_method_name,hp_ID,hyperparameters,communities
0,sample_0,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 1, 3: 2, 4: 0, 5: 0, 6: 5, 7: ..."
1,sample_1,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 2, 3: 2, 4: 0, 5: 0, 6: 3, 7: ..."
2,sample_2,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 2, 3: 2, 4: 0, 5: 0, 6: 3, 7: ..."
3,sample_3,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 2, 3: 3, 4: 0, 5: 0, 6: 4, 7: ..."
4,sample_4,normalized_counts,louvain,louvain_0,"{'parameter1': 5, 'parameter2': 12}","{0: 0, 1: 1, 2: 2, 3: 1, 4: 0, 5: 0, 6: 3, 7: ..."
